In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize = (10,6))
np.set_printoptions(precision = 4, suppress = True)

In [3]:
# Import two datasets
colnames = ["LOAN_ID", "ORIG_CHN", "Seller.Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE","FRST_DTE", "OLTV", 
            "OCLTV", "NUM_BO", "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", "PROP_TYP","NUM_UNIT", "OCC_STAT", 
            "STATE", "ZIP_3", "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"]
Data_A = pd.read_table('D:/1-Intern/Project_1/2018Q1/Acquisition_2018Q1.txt', 
                       sep = '|', header = None, names = colnames)
colnames_P = ["LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat", 
              "Adj.Month.To.Mat", "Maturity.Date", "MSA", "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", 
              "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", "FCC_COST", "PP_COST", "AR_COST", "IE_COST", "TAX_COST", "NS_PROCS",
              "CE_PROCS", "RMW_PROCS", "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"]
Data_P = pd.read_table('D:/1-Intern/Project_1/2018Q1/Performance_2018Q1.txt', 
                       sep = '|', header = None, names = colnames_P,low_memory=False)

In [4]:
df = pd.merge(Data_A.loc[:,["LOAN_ID", "ORIG_CHN", "Seller.Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM","FRST_DTE", "OLTV", 
             "NUM_BO", "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", "PROP_TYP","NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3", "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"]],
              Data_P.loc[:,["LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat", "Adj.Month.To.Mat", "Maturity.Date", "MSA", 
                            "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", "ZB_DTE", "LPI_DTE", "FCC_DTE"]] , on = 'LOAN_ID')

In [ ]:
# Convert string into datetime
from datetime import datetime
df['Monthly.Rpt.Prd'] = [datetime.strptime(x, '%m/%d/%Y') for x in df.loc[:,'Monthly.Rpt.Prd']]

In [52]:
# Check each column's missing value
Number_NA = pd.DataFrame(df.isnull().sum()).unstack()

In [56]:
Number_NA

0  LOAN_ID                      0
   ORIG_CHN                     0
   Seller.Name                  0
   ORIG_RT                      0
   ORIG_AMT                     0
   ORIG_TRM                     0
   FRST_DTE                     0
   OLTV                         0
   NUM_BO                       0
   DTI                       2369
   CSCORE_B                  7301
   FTHB_FLG                     0
   PURPOSE                      0
   PROP_TYP                     0
   NUM_UNIT                     0
   OCC_STAT                     0
   STATE                        0
   ZIP_3                        0
   MI_PCT                 6809722
   Product.Type                 0
   CSCORE_C               5153362
   MI_TYPE                6809722
   RELOCATION_FLG               0
   Monthly.Rpt.Prd              0
   Servicer.Name          8793683
   LAST_RT                      0
   LAST_UPB               2540689
   Loan.Age                     0
   Months.To.Legal.Mat          0
   Adj.Month.T

In [62]:
# coding Delinquncy status into 0,1,2,3,4+ categories
df['Delq.Status_Coding'] = df['Delq.Status'].replace(['4','5','6','7','8','9','10','11','12','13',
                                                      '14','15','16','17','18','19','20','21','22'], '4+')
# Understand how many categories in Delinquency status
df['Delq.Status_Coding'].value_counts(dropna=False)

0      9187791
X        68114
1        52514
2         9927
4+        9424
3         4143
NaN        137
Name: Delq.Status_Coding, dtype: int64

In [63]:
df['Delq.Status'].value_counts(dropna=False)

0      9187791
X        68114
1        52514
2         9927
3         4143
4         2669
5         2016
6         1336
7          941
8          648
9          446
10         375
11         274
12         199
13         158
NaN        137
14         119
15          96
16          51
17          38
18          24
19          16
20          11
21           5
22           2
Name: Delq.Status, dtype: int64

In [61]:
df['Zero.Bal.Code'].value_counts(dropna=False)

NaN     9263800
1.0       67775
6.0         326
9.0          89
2.0          31
16.0         12
3.0          12
15.0          5
Name: Zero.Bal.Code, dtype: int64

In [73]:
#From the below result, we coukd know that only when delinquency status is unknown, Zero Balance code is denoted
df.groupby(['Delq.Status_Coding','Zero.Bal.Code']).size()

Delq.Status_Coding  Zero.Bal.Code
X                   1.0              67775
                    6.0                326
                    16.0                12
dtype: int64

In [7]:
# coding FCC as 'F'
df['FCC_DTE_Coding'] = df['FCC_DTE'].replace(['10/01/2019', '05/01/2019', '08/01/2019', '12/01/2018',
       '04/01/2019', '07/01/2019', '01/01/2019', '09/01/2019',
       '06/01/2019', '12/01/2019', '11/01/2018', '03/01/2019',
       '02/01/2019', '11/01/2019', '10/01/2018', '07/01/2018'],'F')

In [64]:
df['FCC_DTE_Coding'].value_counts(dropna=False)

NaN    9331924
F          126
Name: FCC_DTE_Coding, dtype: int64

In [72]:
# From below result, we know that Foreclosure only shows when loans denoted as 2.0, 3.0, 9.0. 

df.groupby(['FCC_DTE_Coding','Zero.Bal.Code']).size()

FCC_DTE_Coding  Zero.Bal.Code
F               2.0              31
                3.0               9
                9.0              86
dtype: int64

In [74]:
# In addition, when 'Delq.Status_Coding' shows, Foreclosure would not be come up. they totally exlusive. Combined with result above, 
# we could just denote 2.0, 3.0, 9.0. as 'F'
df.groupby(['FCC_DTE_Coding','Delq.Status_Coding']).size()

Series([], dtype: int64)

In [95]:
df['Zero.Bal.Code']=df['Zero.Bal.Code'].replace([2.0, 3.0, 9.0],'F')

In [96]:
df['Zero.Bal.Code'].value_counts(dropna=False)

NaN     9263800
1.0       67775
6.0         326
F           132
16.0         12
15.0          5
Name: Zero.Bal.Code, dtype: int64

In [134]:
df['Delq.Status_Coding'].value_counts()

0      9187791
X        68114
1        52514
2         9927
4+        9424
3         4143
NaN        137
Name: Delq.Status_Coding, dtype: int64

In [135]:
df['Delq.Status_Coding']=df['Delq.Status_Coding'].replace('NaN','N' )

In [142]:
df['Delq.Status_Coding'].value_counts(dropna=False)

0      9187791
X        68114
1        52514
2         9927
4+        9424
3         4143
NaN        137
Name: Delq.Status_Coding, dtype: int64

In [151]:
# Because of deal with missing value, I took a lots of time 
conditions = [
    df['Delq.Status_Coding']== '0' ,
    df['Delq.Status_Coding']== '1',
    df['Delq.Status_Coding']== '2',
    df['Delq.Status_Coding']== '3',
    df['Delq.Status_Coding']== '4+',
    ((pd.isna(df['Delq.Status_Coding']))& (df['Zero.Bal.Code']== 'F' )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 1.0 )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 6.0 )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 16.0 )),
    ((df['Delq.Status_Coding']=='X' )& (pd.isna(df['Zero.Bal.Code']))),
    ((pd.isna(df['Delq.Status_Coding']))& (df['Zero.Bal.Code']== 15.0))
]

In [152]:
choises = ['0','1','2','3','4+','F','P','Other','Other','Other','Other']

In [168]:
df['Loan_Status'] = np.select(condlist=conditions, choicelist = choises)

In [169]:
df['Loan_Status'].value_counts()

0        9187791
P          67775
1          52514
2           9927
4+          9424
3           4143
Other        344
F            132
Name: Loan_Status, dtype: int64

In [158]:
df = df.drop(['LOAN_STATUE','LOAN_STATUE_1'], axis = 1)

In [166]:
df = df.drop(['LOAN_STATUE_NEW'], axis = 1)

In [201]:
Sum_Loan_Status = df.groupby([df['Monthly.Rpt.Prd'], df['Loan_Status']]).size()

In [205]:
Sum_Loan_Status = pd.DataFrame(Sum_Loan_Status)
Sum_Loan_Status= Sum_Loan_Status.unstack()

In [206]:
Sum_Loan_Status


0                                                 
Loan_Status             0       1      2      3     4+     F Other       P
Monthly.Rpt.Prd                                                           
2018-01-01       154450.0   100.0    NaN    NaN    NaN   NaN   NaN     NaN
2018-02-01       293478.0   983.0   10.0    NaN    NaN   NaN   1.0   225.0
2018-03-01       424088.0  1235.0   59.0    2.0    NaN   NaN   2.0   593.0
2018-04-01       422907.0  1381.0  123.0   23.0    2.0   NaN   4.0   944.0
2018-05-01       421498.0  1318.0  147.0   69.0   11.0   NaN   7.0  1387.0
2018-06-01       419646.0  1502.0  153.0   69.0   47.0   NaN  10.0  1616.0
2018-07-01       417628.0  1511.0  174.0   68.0   70.0   1.0   9.0  1956.0
2018-08-01       415403.0  1411.0  202.0   72.0   95.0   NaN  21.0  2247.0
2018-09-01       412488.0  2219.0  294.0  105.0  124.0   NaN  23.0  1930.0
2018-10-01       410372.0  1984.0  345.0  120.0  173.0   2.0  44.0  2190.0
2018-11-01       408156.0  2158.0  402.0  151.0  216.0   3.0  41.0  1867.0
2018-12-01       405594.0  2641.0  459.0  212.0  303.0   3.0  48.0  1823.0
2019-01-01       403855.0  2343.0  554.0  199.0  387.0   7.0  34.0  1830.0
2019-02-01       400800.0  3173.0  528.0  228.0  477.0   5.0  19.0  2108.0
2019-03-01       398457.0  2708.0  511.0  199.0  553.0   3.0  19.0  2756.0
2019-04-01       395359.0  2497.0  523.0  208.0  594.0  11.0   7.0  3229.0
2019-05-01       391512.0  2396.0  530.0  220.0  620.0  10.0   5.0  3888.0
2019-06-01       386809.0  3246.0  642.0  252.0  678.0   6.0   4.0  3641.0
2019-07-01       382442.0  2822.0  674.0  279.0  700.0   9.0  11.0  4690.0
2019-08-01       377214.0  2700.0  671.0  301.0  751.0  14.0   7.0  5259.0
2019-09-01       370711.0  3051.0  713.0  326.0  822.0   8.0   6.0  6000.0
2019-10-01       364083.0  2808.0  719.0  330.0  869.0  20.0   5.0  6788.0
2019-11-01       358099.0  3220.0  729.0  362.0  938.0  17.0   3.0  5441.0
2019-12-01       352742.0  3107.0  765.0  348.0  994.0  13.0  14.0  5367.0